In [64]:
import pandas as pd
import numpy as np

In [90]:
# Load the data
is_factual_data = pd.read_csv('prompts_df_final_predictions.csv')
impersonator_data = pd.read_csv('df_impersonater.csv')
fake_pred_data = pd.read_csv('fake_pred_trained_model_FINAL.csv')


In [91]:
is_factual_data.head()

,prompt,file_name,language,client,assignment_score,similarity_score,is_factually_correct
0,"Hi, I'm Finn Murphy with Tax Residency in Tur...",72K6TSQ829.wav,en,Finn Murphy,90,100.0,True
1,"Hi, this is Fatima Zara Mansuri, a tax reside...",O7O51MZU4N.wav,en,Fatima Zahra Mansouri,86,100.0,True
2,"Hello, Santiago Rivera here, born on 15th Apr...",1PSFC9GSK4.wav,en,Santiago Rivera,90,100.0,True
3,"Good morning, this is Olivia Tremblay with a ...",1JGXELMHOF.wav,en,Olivia Tremblay,90,89.0,True
4,"Hello, this is Isabella Rossi. Social Securit...",VXC2DB72R7.wav,en,Isabella Rossi,90,55.0,False


In [92]:
# how many are is_factually_correct in is_factual_data
is_factual_data['is_factually_correct'].value_counts() 

is_factually_correct
True     200
False    200
Name: count, dtype: int64

In [93]:
# is_factual_data["file_name"] take everything before the ".wav"
is_factual_data["file_name"] = is_factual_data["file_name"].str.split(".").str[0]

# use the file_name, is_factually_correct columns
is_factual_data = is_factual_data[["file_name", "is_factually_correct"]]
print(is_factual_data.shape)
is_factual_data.head()

(400, 2)


,file_name,is_factually_correct
0,72K6TSQ829,True
1,O7O51MZU4N,True
2,1PSFC9GSK4,True
3,1JGXELMHOF,True
4,VXC2DB72R7,False


In [94]:
impersonator_data.head()

impersonator_data["is_impersonator"] = impersonator_data["score"] < 0.7
impersonator_data.head()

,file_name,ref_name,score,prediction,is_impersonator
0,72K6TSQ829,ZGZHPG1TS8,0.615426,1,True
1,O7O51MZU4N,YDDKTODLGX,0.645373,1,True
2,1PSFC9GSK4,H01FH3KEY8,-0.034738,0,True
3,1JGXELMHOF,CBDX295MEZ,0.551791,1,True
4,VXC2DB72R7,2IM42LTT5R,0.636094,1,True


In [95]:
# how many predictions are True
impersonator_data["is_impersonator"].value_counts()

is_impersonator
True     244
False    156
Name: count, dtype: int64

In [96]:
# only keep the file_name, is_impersonator columns
impersonator_data = impersonator_data[["file_name", "is_impersonator"]]
print(impersonator_data.shape)
impersonator_data.head()

(400, 2)


,file_name,is_impersonator
0,72K6TSQ829,True
1,O7O51MZU4N,True
2,1PSFC9GSK4,True
3,1JGXELMHOF,True
4,VXC2DB72R7,True


In [97]:
fake_pred_data.head()

# fake_pred_data["is_fake"] = True if Predicted_label_avg < 0.5
fake_pred_data["is_fake"] = fake_pred_data["Predicted_label_avg"] < 0.5
fake_pred_data.head()


,Name,Predicted_Labels_34,Predicted_Labels_808,Predicted_Labels_999,Predicted_label_96,Predicted_label_200,Predicted_label_avg,Probabilities,Probabilities_0,Probabilities_1,is_fake
0,72K6TSQ829,0,0,0,0,0,0.0,[0.85575485 0.1442451 ],0.855755,0.144245,True
1,O7O51MZU4N,0,0,0,0,0,0.0,[0.8609644 0.13903558],0.860964,0.139036,True
2,1PSFC9GSK4,1,1,1,1,1,1.0,[0.04502322 0.9549768 ],0.045023,0.954977,False
3,1JGXELMHOF,0,0,0,0,0,0.0,[0.9161273 0.08387267],0.916127,0.083873,True
4,VXC2DB72R7,1,0,1,1,1,0.8,[0.47609314 0.52390695],0.476093,0.523907,False


In [98]:
# how many predictions are True
fake_pred_data["is_fake"].value_counts()

is_fake
False    225
True     175
Name: count, dtype: int64

In [99]:
# only keep the file_name, is_fake columns
fake_pred_data = fake_pred_data[["Name", "is_fake"]]
print(fake_pred_data.shape)
fake_pred_data.head()

(400, 2)


,Name,is_fake
0,72K6TSQ829,True
1,O7O51MZU4N,True
2,1PSFC9GSK4,False
3,1JGXELMHOF,True
4,VXC2DB72R7,False


In [100]:
# rename the Name column to file_name
fake_pred_data.rename(columns={"Name": "file_name"}, inplace=True)
fake_pred_data.head()

,file_name,is_fake
0,72K6TSQ829,True
1,O7O51MZU4N,True
2,1PSFC9GSK4,False
3,1JGXELMHOF,True
4,VXC2DB72R7,False


In [101]:

# merge the dataframes by the file_name column, order: is_fake, is_factually_correct, is_impersonator

merged_data = pd.merge(fake_pred_data, is_factual_data, on="file_name", how="inner")
merged_data = pd.merge(merged_data, impersonator_data, on="file_name", how="inner")

print(merged_data.shape)
merged_data.head()


(400, 4)


,file_name,is_fake,is_factually_correct,is_impersonator
0,72K6TSQ829,True,True,True
1,O7O51MZU4N,True,True,True
2,1PSFC9GSK4,False,True,True
3,1JGXELMHOF,True,True,True
4,VXC2DB72R7,False,False,True


In [102]:
# is there any missing data?
merged_data.isnull().sum()

file_name               0
is_fake                 0
is_factually_correct    0
is_impersonator         0
dtype: int64

In [103]:
# show the distribution of the labels in the merged data for each column
for col in merged_data.columns[1:]:
    print(merged_data[col].value_counts())


is_fake
False    225
True     175
Name: count, dtype: int64
is_factually_correct
True     200
False    200
Name: count, dtype: int64
is_impersonator
True     244
False    156
Name: count, dtype: int64


In [104]:
# rename file_name column to rec_id
merged_data.rename(columns={"file_name": "rec_id"}, inplace=True)

merged_data

,rec_id,is_fake,is_factually_correct,is_impersonator
0,72K6TSQ829,True,True,True
1,O7O51MZU4N,True,True,True
2,1PSFC9GSK4,False,True,True
3,1JGXELMHOF,True,True,True
4,VXC2DB72R7,False,False,True
...,...,...,...,...
395,FPMY3OD663,True,False,False
396,TKWJHEYRLP,False,True,True
397,XATHQAOSPE,False,False,True
398,FGQ1PL8LA6,False,False,True


In [105]:
# save the data
merged_data.to_csv("../deep_snoopy.csv", index=False)

In [106]:
# count the combinations of the labels of is_fake, is_factually_correct, is_impersonator

merged_data.groupby(["is_fake", "is_factually_correct", "is_impersonator"]).size()


is_fake  is_factually_correct  is_impersonator
False    False                 False              23
                               True               89
         True                  False              88
                               True               25
True     False                 False              25
                               True               63
         True                  False              20
                               True               67
dtype: int64

In [107]:
# count the cases where is_fake and is_impersonator are simultaneously True

merged_data[(merged_data["is_fake"] == True) & (merged_data["is_impersonator"] == True)].shape

(130, 4)